In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from utils import prep_data
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.impute import KNNImputer,SimpleImputer

In [ ]:
DATA_FOLDER = Path("./cedas2025_material/data")

In [ ]:
data = pd.read_parquet(DATA_FOLDER / "chargecurves_train.parquet")
# val = pd.read_parquet(DATA_FOLDER  / "chargecurves_validation_holdback.parquet")

In [ ]:
def create_dummy_predictions(df: pd.DataFrame):
    df["power"] = df["power"].replace(np.nan, 1)
    df.to_parquet("val_preds.parquet")

In [ ]:
data

In [ ]:
data = data.drop("timestamp", axis=1)
data_X, data_y = data[["id", "soc", "nominal_power", "location_id", "sub_id"]], data[["power"]]

In [ ]:
# test = data.groupby("id").transform(pd.DataFrame.interpolate
# test = data
test = data.groupby('id')['soc'].apply(lambda x : x.interpolate(method = "spline", order = 1, limit_direction = "both"))
# for i in range(len(test.id.unique())):
#   mask = test.loc[:,'id']==test.id.unique()[i]
#   test[mask]=test[mask].interpolate(method='index', limit_direction="both")

In [ ]:
# test[test["soc"].isna()]
interpolated = test.reset_index().drop(["id", "level_1"], axis=1)

In [ ]:
interpolated["soc"]

In [ ]:
data[data["soc"] != interpolated["soc"]

In [ ]:
train_size = int(len(data) * .8)
train_X, train_y = data_X[:train_size], data_y[:train_size]
val_X, val_y = data_X[train_size:], data_y[train_size:]

In [ ]:
data[data["id"] == 1764]

In [ ]:
train_X[train_X["soc"].isna()]

In [ ]:
data[data["power"].isna() & data["soc"].isna()]["id"].value_counts()

In [ ]:
data["power"].dropna()

In [ ]:
data["power"].describe()